Created by TIanjing Cao & Wenqing Gao

In [2]:
pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install finrl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 234 kB 33.0 MB/s 
     |████████████████████████████████| 152 kB 48.6 MB/s 
     |████████████████████████████████| 177 kB 42.9 MB/s 
     |████████████████████████████████| 54.5 MB 2.4 MB/s 
     |████████████████████████████████| 2.3 MB 39.6 MB/s 
     |████████████████████████████████| 199 kB 47.5 MB/s 
     |████████████████████████████████| 1.2 MB 35.1 MB/s 
     |████████████████████████████████| 153 kB 44.1 MB/s 
     |████████████████████████████████| 952 kB 47.8 MB/s 
     |████████████████████████████████| 125 kB 46.6 MB/s 
     |████████████████████████████████| 4.1 MB 40.6 MB/s 
     |████████████████████████████████| 271 kB 52.9 MB/s 
     |████████████████████████████████| 288 kB 31.3 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |███████████████████████████

In [5]:
pip install PyPortfolioOpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 3.2 MB/s 


In [1]:
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
import datetime
import random
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn
from sklearn.datasets import make_sparse_spd_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy.stats import random_correlation
import scipy.optimize as optimize
import scipy.interpolate as sci
import time
import tqdm

import import_ipynb

import tensorflow as tf
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from hyperopt import rand
import sys
import math

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
import itertools

import matplotlib.pyplot as plt
from numpy import linalg as la
from cvxpy import *

from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import black_litterman, risk_models

c:\Users\xiang\anaconda3\envs\RLcapstone\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class compare_strategy:
  def __init__(self, data, mvo_start_date, interest_rate, min_return):
      self.start_date = mvo_start_date
      self.interest_rate = interest_rate
      self.min_return = min_return
      self.price = data.get_data().dropna()
      self.shift = data.get_shift() 
      self.shift_returns = data.get_shift_returns()
      self.shift_returns_mean, self.shift_returns_var, self.covariance = data.get_mean_var_cov()
      self.StockList = self.price.columns.values.tolist() + ["InterestRate"]

      self.index = self.shift_returns.index
      self.start_index = self.index.get_loc(self.start_date)
      self.end_index= self.index.get_loc(self.shift_returns.index[-1])
  
  def set_start_date(self, start_date):
      self.start_date = start_date
      self.start_index = self.index.get_loc(self.start_date)


  def MarkowitzOpt(self, mean, variance, covariance, interest_rate, min_return):
      n = mean.size + 1                   # Number of assets (number of stocks + interest rate)
      
      mu = mean.values                    # Mean returns of n assets
      temp = np.full(n, interest_rate)
      temp[:-1] = mu
      mu = temp
          
      counter = 0
      Sigma = np.zeros((n,n))                 # Covariance of n assets
      for i in np.arange(n-1):
          for j in np.arange(i, n-1):
              if i==j:
                  Sigma[i,j] = variance[i]
              else:
                  Sigma[i,j] = covariance[counter]
                  Sigma[j,i] = Sigma[i,j]
                  counter+=1
      Sigma = self.nearestPD(Sigma)                # Converting covariance to the nearest positive-definite matrix
      
      # Ensuring feasability of inequality contraint
      if mu.max() < min_return:
          min_return = interest_rate
      
      w = Variable(n)                         # Portfolio allocation vector
      ret = mu.T*	w
      risk = quad_form(w, Sigma)
      min_ret = Parameter(nonneg=True)
      min_ret.value = min_return
      prob = Problem(Minimize(risk),          # Restricting to long-only portfolio
                    [ret >= min_ret,
                    sum(w) == 1,
                    w >= 0])
      prob.solve()
      return w.value

  def nearestPD(self, A):
      """Find the nearest positive-definite matrix to input
      A Python/Numpy port of John D'Errico's `nearestSPD` MATLAB code [1], which
      credits [2].
      [1] https://www.mathworks.com/matlabcentral/fileexchange/42885-nearestspd
      [2] N.J. Higham, "Computing a nearest symmetric positive semidefinite
      matrix" (1988): https://doi.org/10.1016/0024-3795(88)90223-6
      """

      B = (A + A.T) / 2
      _, s, V = la.svd(B)

      H = np.dot(V.T, np.dot(np.diag(s), V))

      A2 = (B + H) / 2

      A3 = (A2 + A2.T) / 2

      if self.isPD(A3):
          return A3

      spacing = np.spacing(la.norm(A))
      # The above is different from [1]. It appears that MATLAB's `chol` Cholesky
      # decomposition will accept matrixes with exactly 0-eigenvalue, whereas
      # Numpy's will not. So where [1] uses `eps(mineig)` (where `eps` is Matlab
      # for `np.spacing`), we use the above definition. CAVEAT: our `spacing`
      # will be much larger than [1]'s `eps(mineig)`, since `mineig` is usually on
      # the order of 1e-16, and `eps(1e-16)` is on the order of 1e-34, whereas
      # `spacing` will, for Gaussian random matrixes of small dimension, be on
      # othe order of 1e-16. In practice, both ways converge, as the unit test
      # below suggests.
      I = np.eye(A.shape[0])
      k = 1
      while not self.isPD(A3):
          mineig = np.min(np.real(la.eigvals(A3)))
          A3 += I * (-mineig * k**2 + spacing)
          k += 1

      return A3

  def isPD(self, B):
      """Returns true when input is positive-definite, via Cholesky"""
      try:
          _ = la.cholesky(B)
          return True
      except la.LinAlgError:
          return False

  def blacklitter(self, S, start_index):
      viewdict = {"^GSPC": 0.01, '^DJI':0.02} #,
      #S = risk_models.sample_cov(prices_df.iloc[start_index:start_index+20])
      bl = black_litterman.BlackLittermanModel(S, pi="equal",absolute_views = viewdict, omega="default")
      weight = bl.optimize(0.7)
      weights = []
      for key,value in weight.items():
          weights.append(value)
      weights.append(0)
      
      return weights
  
  def Markowitz(self):      
      # Variable Initialization
      # index = self.shift_returns.index
      # start_index = index.get_loc(self.start_date)
      # end_date = index[-1]
      # end_index = index.get_loc(end_date)
      date_index_iter = self.start_index
      # StockList = self.price.columns.values.tolist()
      # StockList.append('InterestRate')
      distribution = pd.DataFrame(index=self.StockList)
      returns = pd.Series(index=self.index)
      
      # Start Value
      total_value = 1.0
      returns[self.index[date_index_iter]] = total_value
      portfolio_alloc = np.array([1/(self.price.shape[1]+1)]*(self.price.shape[1]+1))
      
      while date_index_iter + self.shift < self.end_index:
          # Calculating portfolio return
          date = self.index[date_index_iter]
          portfolio_alloc = self.MarkowitzOpt(self.shift_returns_mean.loc[date], self.shift_returns_var.loc[date], self.covariance.loc[date], self.interest_rate, self.min_return)
          # print(portfolio_alloc)
          distribution[date] = portfolio_alloc

          # print(date_index_iter, portfolio_alloc, type(portfolio_alloc))
          # print(self.shift)

          total_value_temp = total_value
          date1 = self.index[date_index_iter]
          for i in range(self.shift):
            
            date2 = self.index[date_index_iter+1]
            temp1 = self.price.loc[date2]/self.price.loc[date1]
            temp1.loc[self.StockList[-1]] = self.interest_rate+1
            temp2 = pd.Series(np.array(portfolio_alloc.ravel()).reshape(len(portfolio_alloc)),index=self.StockList)
            date_index_iter += 1
            total_value_temp = np.sum(total_value*temp2*temp1)
            returns[self.index[date_index_iter]] = total_value_temp
          total_value=total_value_temp
            # Increment Date
            # date_index_iter += 1
            # returns[self.index[date_index_iter]] = total_value_temp
            # print(date1, date2, total_value)

          # Calculating portfolio return
          # date2 = self.index[date_index_iter+self.shift]
          # temp1 = self.price.loc[date2]/self.price.loc[date]
          # temp1.loc[self.StockList[-1]] = self.interest_rate+1
          # temp2 = pd.Series(np.array(portfolio_alloc.ravel()).reshape(len(portfolio_alloc)),index=self.StockList)
          # total_value = np.sum(total_value*temp2*temp1)
          # # Increment Date
          # date_index_iter += self.shift
          # returns[self.index[date_index_iter]] = total_value

      # Remove dates that there are no trades from returns

      date1 = self.index[date_index_iter]
      while date_index_iter < self.end_index:
        date2 = self.index[date_index_iter+1]
        temp1 = self.price.loc[date2]/self.price.loc[date1]
        temp1.loc[self.StockList[-1]] = self.interest_rate+1
        temp2 = pd.Series(np.array(portfolio_alloc.ravel()).reshape(len(portfolio_alloc)),index=self.StockList)
        date_index_iter += 1
        total_value_temp = np.sum(total_value*temp2*temp1)
        returns[self.index[date_index_iter]] = total_value_temp
      returns = returns[np.isfinite(returns)]
      return returns

  def bl_model(self):
      date_index_iter = self.start_index
      distribution = pd.DataFrame(index=self.StockList)
      returns_bl = pd.Series(index=self.index)

      # Start Value
      total_value = 1.0
      returns_bl[self.index[date_index_iter]] = total_value
      portfolio_alloc = np.array([1/(self.price.shape[1]+1)]*(self.price.shape[1]+1))
      # print(portfolio_alloc)
      
      while date_index_iter + self.shift < self.end_index: #+ self.shift
          # date = self.index[date_index_iter]
          total_value_temp = total_value
          date1 = self.index[date_index_iter]
          for i in range(self.shift):
            
            # date1 = self.index[date_index_iter]
            date2 = self.index[date_index_iter+1]
            temp1 = self.price.loc[date2]/self.price.loc[date1]
            temp1.loc[self.StockList[-1]] = self.interest_rate+1
            temp2 = pd.Series(np.array(portfolio_alloc.ravel()).reshape(len(portfolio_alloc)),index=self.StockList)
            date_index_iter += 1
            total_value_temp = np.sum(total_value*temp2*temp1)
            returns_bl[self.index[date_index_iter]] = total_value_temp
          total_value = total_value_temp
            # total_value = np.sum(total_value*temp2*temp1)
            # Increment Date
            
            # returns_bl[self.index[date_index_iter]] = total_value


          # date = self.index[date_index_iter]

          S = risk_models.sample_cov(self.price.iloc[date_index_iter-self.shift:date_index_iter])

          """try:
            #portfolio_alloc = np.array(blacklitter(nearestPD(S), date_index_iter))
            try:
              portfolio_alloc = np.array(blacklitter(nearestPD(S), date_index_iter))
            except:
              portfolio_alloc = np.array(blacklitter(S, date_index_iter))
          except:
            portfolio_alloc = portfolio_alloc"""

          try:
            portfolio_alloc = np.array(self.blacklitter(self.nearestPD(S), date_index_iter-self.shif))
          except:
            try:
              portfolio_alloc = np.array(self.blacklitter(S, date_index_iter-self.shif))
            except:
              portfolio_alloc = portfolio_alloc

          

            
      date1 = self.index[date_index_iter]
      while date_index_iter < self.end_index:
        date2 = self.index[date_index_iter+1]
        temp1 = self.price.loc[date2]/self.price.loc[date1]
        temp1.loc[self.StockList[-1]] = self.interest_rate+1
        temp2 = pd.Series(np.array(portfolio_alloc.ravel()).reshape(len(portfolio_alloc)),index=self.StockList)
        date_index_iter += 1
        total_value_temp = np.sum(total_value*temp2*temp1)
        returns_bl[self.index[date_index_iter]] = total_value_temp


      returns_bl = returns_bl[np.isfinite(returns_bl)]

      return returns_bl

In [40]:
index_list = ['^GSPC', '^DJI', '^IXIC', '^HSI', 'GC=F', '000001.SS',  '^RUT', '^N100']
data2 = Data(index_list, start_date='2002-01-01', end_date='2022-06-30', n_past=10, mvo_shift=20)
compare=compare_strategy(data2,'2019-06-10',interest_rate=0.03/12, min_return = 0.03)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (41042, 8)


In [42]:
bl_returns = compare.bl_model()
bl_returns

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:188: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


date
2019-06-10    1.000000
2019-06-11    1.004229
2019-06-12    1.000851
2019-06-13    1.004078
2019-06-14    1.000106
                ...   
2022-06-23    1.193146
2022-06-24    1.216823
2022-06-27    1.220478
2022-06-28    1.212521
2022-06-29    1.205473
Length: 796, dtype: float64

In [41]:
mvo_returns = compare.Markowitz()
mvo_returns

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:127: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 41 times so far.

  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.7/dist-packages/cvxpy/expressions/expression.py:593: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplic

date
2019-06-10    1.000000
2019-06-11    1.002500
2019-06-12    1.002500
2019-06-13    1.002500
2019-06-14    1.002500
                ...   
2022-06-23    1.006758
2022-06-24    1.006758
2022-06-27    1.006758
2022-06-28    1.006758
2022-06-29    1.006758
Length: 796, dtype: float64

In [54]:
data2 = Data(index_list, start_date='2002-01-01', end_date='2022-06-30', n_past=10, mvo_shift=20)
data2.generate_df()
data2.prices

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (41042, 8)


,^GSPC,^DJI,^IXIC,^HSI,GC=F,000001.SS,^RUT,^N100
date,,,,,,,,
2002-01-02,1154.670044,10073.400391,1979.250000,11350.849609,278.899994,NaN,487.190002,789.780029
2002-01-03,1165.270020,10172.139648,2044.270020,11423.519531,278.200012,NaN,495.510010,803.770020
2002-01-04,1172.510010,10259.740234,2059.379883,11702.150391,278.899994,1611.392944,499.299988,803.099976
2002-01-07,1164.890015,10197.049805,2037.099976,11892.639648,278.600006,1596.760010,493.179993,794.780029
2002-01-08,1160.709961,10150.549805,2055.739990,11713.709961,278.899994,1583.461060,497.899994,787.419983
...,...,...,...,...,...,...,...,...
2022-06-23,3795.729980,30677.359375,11232.190430,21273.869141,1825.699951,3320.149902,1711.670044,1123.689941
2022-06-24,3911.739990,31500.679688,11607.620117,21719.060547,1826.500000,3349.750000,1765.739990,1157.680054
2022-06-27,3900.110107,31438.259766,11524.549805,22229.519531,1820.900024,3379.189941,1771.739990,1164.479980


In [11]:
compare.price

,^GSPC,^DJI,^IXIC,^HSI,GC=F,000001.SS,^RUT,^N100
date,,,,,,,,
2002-01-04,1172.510010,10259.740234,2059.379883,11702.150391,278.899994,1611.392944,499.299988,803.099976
2002-01-07,1164.890015,10197.049805,2037.099976,11892.639648,278.600006,1596.760010,493.179993,794.780029
2002-01-08,1160.709961,10150.549805,2055.739990,11713.709961,278.899994,1583.461060,497.899994,787.419983
2002-01-09,1155.140015,10094.089844,2044.890015,11440.719727,283.399994,1561.348999,494.739990,791.750000
2002-01-10,1156.550049,10067.860352,2047.239990,11256.070312,287.100006,1576.437012,495.309998,780.460022
...,...,...,...,...,...,...,...,...
2022-06-23,3795.729980,30677.359375,11232.190430,21273.869141,1825.699951,3320.149902,1711.670044,1123.689941
2022-06-24,3911.739990,31500.679688,11607.620117,21719.060547,1826.500000,3349.750000,1765.739990,1157.680054
2022-06-27,3900.110107,31438.259766,11524.549805,22229.519531,1820.900024,3379.189941,1771.739990,1164.479980


In [20]:
compare.set_start_date("2012-06-08")